# Experimental new pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline
import multiprocessing

## Load analysis and project

In [3]:
analysis = Analysis.objects.get_or_create(id=1)[0]

DEBUG 2016-09-21 16:35:19,075 utils.py:91 | (0.000) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2016-09-21 16:35:19,076 utils.py:91 | (0.000) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 1 LIMIT 21; args=(1,)


In [4]:
project = Project.objects.get_or_create(id=1)[0]

DEBUG 2016-09-21 16:35:19,108 utils.py:91 | (0.000) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 1 LIMIT 21; args=(1,)


## Get files etc

In [5]:
# experiment = analysis.experiment
# print experiment.id, experiment

In [6]:
# comparisons = experiment.comparison_set.all()
# for comparison in comparisons:
#     print comparison, attributes
#     attributes = Attribute.objects.filter(comparison__id = comparison.id)
#     for attribute in attributes:
#         samples = Sample.objects.filter(attribute__id = attribute.id)
#         print samples

## Start the pipeline

In [7]:
pipeline = Rpy2Pipeline(analysis, project, False)

******************************************
Setup rpy2 connection
******************************************
[1] "Initialising PuMP logging"
[1] "PuMP log level set to DEBUG"
2016-09-21 16:35:22 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2016-09-21 16:35:22 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2016-09-21 16:35:22 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2016-09-21 16:35:22 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2016-09-21 16:35:23 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-21 16:35:23 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx512m
2016-09-21 16:35:23 INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-21 16:35:23 INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/

In [8]:
pipeline.setup()

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-21 16:35:23,957 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)



Setup called
2016-09-21 16:35:23 INFO:Pimp.getPimpWd:Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
2016-09-21 16:35:23 INFO:Pimp.getPimpWd:Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1
2016-09-21 16:35:23 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 1
2016-09-21 16:35:23 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-21 16:35:23 1:INFO:Pimp.validateInput:Input validated correctly!
2016-09-21 16:35:23 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 1
2016-09-21 16:35:23 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set


WARNING 2016-09-21 16:35:23,983 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_1

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2016-09-21 16:35:24 1:INFO:Pimp.getAnalysisParams:Setting params
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: ppm Value: 3
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: mindetections Value: 3
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: minintensity Value: 5000
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: rtwindow Value: 0.05
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: rt.alignment Value: NA
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2016-09-21 16:35:24 1:DEBUG:Pimp.getAnalysisParams:Name: rtwindow Value: 0.05
2016-09-21 16:35:24 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 1
2016-09-21 16:35:24 1:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set


WARNING 2016-09-21 16:35:24,203 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_1

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2016-09-21 16:35:24 1:INFO:Pimp.createAnalysisDir:Analysis dir created at analysis_1
2016-09-21 16:35:24 1:INFO:Pimp.stds.createAnnotationFile:Creating annotation file from calibration_samples/standard/Std1_1_20150422_150810.csv,calibration_samples/standard/Std2_1_20150422_150711.csv,calibration_samples/standard/Std3_1_20150422_150553.csv, outputting to analysis_1/stds_db.xml
2016-09-21 16:35:24 1:INFO:Pimp.stds.createAnnotationFile.parseStandardsFiles:Parsing standards files
2016-09-21 16:35:24 1:INFO:Pimp.stds.createAnnotationFile.determineFileParser:Parser for calibration_samples/standard/Std1_1_20150422_150810.csv is .parse.toxid.file
2016-09-21 16:35:24 1:INFO:Pimp.stds.createAnnotationFile.parse.toxid.file:Parsing toxid file: calibration_samples/standard/Std1_1_20150422_150810.csv
2016-09-21 16:35:24 1:INFO:Pimp.stds.createAnnotationFile.determineFileParser:Parser for calibration_samples/standard/Std2_1_20150422_150711.csv is .parse.toxid.file
2016-09-21 16:35:24 1:INFO:Pimp.stds

In [9]:
xcms_params = pipeline.get_value(pipeline.pimp_params, 'xcms.params')            
mzmatch_params = pipeline.get_value(pipeline.pimp_params, 'mzmatch.params')
peakml_params = pipeline.get_value(pipeline.pimp_params, 'peakml.params')
mzmatch_outputs = pipeline.get_value(pipeline.pimp_params, 'mzmatch.outputs')
mzmatch_filters = pipeline.get_value(pipeline.pimp_params, 'mzmatch.filters')
n_slaves = multiprocessing.cpu_count()

In [10]:
raw_data_dict = {}
groups_dict = {}
polarity_pos, polarity_neg = pipeline.metadata.files.keys()

## Process positive raw data

In [11]:
raw_data_pos, groups_pos = pipeline.process_raw_data(polarity_pos, xcms_params, mzmatch_params, 
                                         peakml_params, mzmatch_outputs, mzmatch_filters, n_slaves)

Creating /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/positive/combined
------------------------------------------------
positive /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/positive /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/positive/combined
------------------------------------------------
[beer_smell with 2 levels, beer_colour with 2 levels, beer_taste with 3 levels]
beer_smell
 - smell_good ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3', 'Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - smell_bad ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_colour
 - colour_dark ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - colour_light ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3', 'Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_taste
 - taste_delicious ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_

In [12]:
raw_data_dict['positive'] = raw_data_pos
groups_dict['positive'] = groups_pos

## Process negative raw data

In [13]:
raw_data_neg, groups_neg = pipeline.process_raw_data(polarity_neg, xcms_params, mzmatch_params, 
                                         peakml_params, mzmatch_outputs, mzmatch_filters, n_slaves)

Creating /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/negative/combined
------------------------------------------------
negative /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/negative /Users/joewandy/git/pimp/django_projects/pimp_data/projects/1/analysis_1/negative/combined
------------------------------------------------
[beer_smell with 2 levels, beer_colour with 2 levels, beer_taste with 3 levels]
beer_smell
 - smell_good ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3', 'Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - smell_bad ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_colour
 - colour_dark ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - colour_light ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3', 'Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_taste
 - taste_delicious ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_

In [14]:
raw_data_dict['negative'] = raw_data_neg
groups_dict['negative'] = groups_neg

### Run stats

In [15]:
print groups_dict['positive']

$group_0
[1] "Beer_1_full2" "Beer_1_full3" "Beer_1_full1" "Beer_2_full1" "Beer_2_full3"
[6] "Beer_2_full2"

$group_11
[1] "Beer_4_full3" "Beer_4_full2" "Beer_4_full1"

$group_4
[1] "Beer_3_full1" "Beer_3_full2" "Beer_3_full3"




In [16]:
print groups_dict['negative']

$group_0
[1] "Beer_1_full2" "Beer_1_full3" "Beer_1_full1" "Beer_2_full1" "Beer_2_full3"
[6] "Beer_2_full2"

$group_11
[1] "Beer_4_full3" "Beer_4_full2" "Beer_4_full1"

$group_4
[1] "Beer_3_full1" "Beer_3_full2" "Beer_3_full3"




In [17]:
save_fixtures = True
pipeline.run_stats(raw_data_dict, pipeline.analysis.id, groups_dict, pipeline.metadata.r_groups,
                   pipeline.metadata.r_contrasts, pipeline.metadata.r_names, 
                   pipeline.metadata.r_databases, pipeline.metadata.r_dbs, 
                   mzmatch_outputs, save_fixtures, pipeline.working_dir)

['Beer_1_full2', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_1_full3', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_3_full2', 'smell_good', 'colour_light', 'taste_okay', 'sample']
['Beer_1_full1', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_3_full3', 'smell_good', 'colour_light', 'taste_okay', 'sample']
['Beer_4_full3', 'smell_bad', 'colour_light', 'taste_awful', 'sample']
['Beer_3_full1', 'smell_good', 'colour_light', 'taste_okay', 'sample']
['Beer_4_full1', 'smell_bad', 'colour_light', 'taste_awful', 'sample']
['Beer_2_full1', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_2_full3', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_2_full2', 'smell_good', 'colour_dark', 'taste_delicious', 'sample']
['Beer_4_full2', 'smell_bad', 'colour_light', 'taste_awful', 'sample']
